## Đồ án học máy sử dụng Apache Spark ML và SparkSQL
* Đồ án này sử dụng pyspark phiên bản 3.4.0 và được chạy trên Jupyter Notebook

## 1. Nhập dữ liệu và mô tả dữ liệu

#### Load dữ liệu sử dụng một lược đồ tạo thủ công
* Trong notebook này, dữ liệu sử dụng đó là dữ liệu chứa chi tiết của các chuyến bay
* Ở những bước đầu tiên, nhóm thực hiện khám phá dữ liệu sau khi load nó vào DataFrame


In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkMLExample').getOrCreate()

In [30]:
# import the dataframe sql data types
from pyspark.sql.types import *
#
# flightSchema describes the structure of the data in the flights.csv file
#
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])
#
# Use the dataframe reader to read the file and 
#
flights = spark.read.csv('data/raw-flight-data.csv', schema=flightSchema, header=True)
flights.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

### Load dữ liệu sử dụng tính năng tự động tạo lược đồ
* Nếu không định nghĩa sẵn lược đồ, có thể cho Spark đọc file và tạo schema tự động
* Để minh hoạ, nhóm chọn tập dữ liệu `airports.csv` vì tính đơn giản của nó

In [31]:
airports = spark.read.csv('data/airports.csv', header=True, inferSchema=True)
airports.show(10)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
+----------+-----------+-----+--------------------+
only showing top 10 rows



Lược đồ được tạo tự động từ Spark:

In [32]:
# Show the inferred schema for the airports dataframe
airports.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)



### Sử dụng các method có sẵn trong DataFrame
Spark DataFrames cung cấp nhiều hàm có sẵn dùng để trích xuất và xử lý dữ liệu.  
Dưới đây là ví dụ dùng để hiển thị 5 thành phố đầu tiên trong tập dữ liệu về sân bay

In [33]:
cities = airports.select("city", "name")
cities.limit(5).show()

+-----------+--------------------+
|       city|                name|
+-----------+--------------------+
|Adak Island|                Adak|
|  Anchorage|Ted Stevens Ancho...|
|      Aniak|       Aniak Airport|
|     Barrow|Wiley Post/Will R...|
|     Bethel|      Bethel Airport|
+-----------+--------------------+



### Minh hoạ một số toán tử
Toán tử trong SparkSQL được sử dụng tương tự như trong SQL.  
Ở đây, sử dụng toán tử JOIN để kết hợp bảng flights và airports, sau đó sử dụng GROUP BY và COUNT để đếm số chuyến bay từ mỗi sân bay.  
Sau đó, dùng ORDERBY và LIMIT để hiện ra top 5 sân bay dựa trên tổng số chuyến bay

In [34]:
from pyspark.sql import functions as F

flightsByOrigin = flights\
.join(airports, flights.OriginAirportID == airports.airport_id)\
.groupBy("city")\
.agg(F.count(F.lit(1)).alias("Count"))\
.orderBy("Count", ascending=False)

flightsByOrigin.limit(5).show()

+-----------------+------+
|             city| Count|
+-----------------+------+
|          Chicago|177845|
|          Atlanta|149970|
|      Los Angeles|118684|
|         New York|118540|
|Dallas/Fort Worth|105024|
+-----------------+------+



### Mô tả thống kê của dữ liệu
Trong đề tài này, dữ liệu dùng để huấn luyện được lấy từ tập flights. Do đó, để biết thấu hiểu dữ liệu một cách kỹ càng, thực hiện mô tả các giá trị thống kê trong dữ liệu bằng hàm describe:  
Hàm này sẽ hiển thị các giá trị bao gồm count, mean, stddev, min, max tương ứng với số lượng, trung bình, độ lệch chuẩn, min, max của mỗi cột dữ liệu

In [35]:
flights.describe().show()

+-------+-----------------+------------------+-------+------------------+------------------+-----------------+-----------------+
|summary|       DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|         DepDelay|         ArrDelay|
+-------+-----------------+------------------+-------+------------------+------------------+-----------------+-----------------+
|  count|          2719418|           2719418|2719418|           2719418|           2719418|          2691974|          2690385|
|   mean|15.79747468024408|3.8983907586108497|   null| 12742.26441172339|12742.455345592329|10.53686662649788| 6.63768791455498|
| stddev|8.799860168985404|1.9859881390373355|   null|1501.9729397025644|1501.9692528927876|36.09952806643144|38.64881489390081|
|    min|                1|                 1|     9E|             10140|             10140|              -63|              -94|
|    max|               31|                 7|     YV|             15376|             15376|     

## 2. Làm sạch dữ liệu và khám phá dữ liệu

### Loại bỏ các dòng lặp lại
Ở đây sử dụng `dropDuplicates` để loại bỏ dòng lặp lại

In [36]:
total_flights = flights.count()
unique_flights = flights.dropDuplicates().count()

print("Number of duplicate rows = ",total_flights - unique_flights)

Number of duplicate rows =  22435


### Tìm các dữ liệu `null` và loại bỏ, lấp đầy chúng
Ở đây sử dụng `dropna` để loại bỏ các chuyến bay không có dữ liệu gì cả

In [37]:
unique_flights_withoutNA =  flights.dropDuplicates()\
.dropna(how="any", subset=["ArrDelay", "DepDelay"]).count()

print("Missing values (excluding dups) = ", total_flights - unique_flights_withoutNA)

Missing values (excluding dups) =  46233


### Làm sạch dữ liệu
Tiếp theo, dùng fillna để lấp đầy các dòng null ở cột ArrDelay và DepDelay thành giá trị 0

In [38]:
data = flights.dropDuplicates().fillna(value=0, subset=["ArrDelay", "DepDelay"]).repartition(32)

# Let's cache this for efficient future use
data.cache()

print("Number of rows in cleaned data set = ", data.count(), "Number of partitions = ", data.rdd.getNumPartitions())

Number of rows in cleaned data set =  2696983 Number of partitions =  32


### Kiểm tra lại giá trị thống kê

Sau khi làm sạch dữ liệu, kiểm tra lại dòng count ở bảng thống kê. Nếu thấy tất cả bằng nhau, nghĩa là không còn các giá trị null không cần thiết

In [39]:
data.describe().show()

+-------+------------------+------------------+-------+------------------+-----------------+------------------+------------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|    DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+------------------+-------+------------------+-----------------+------------------+------------------+
|  count|           2696983|           2696983|2696983|           2696983|          2696983|           2696983|           2696983|
|   mean|15.798996508320593| 3.900369412784582|   null|12742.459424846207|12742.85937657004|10.531134234068217|6.6679285705545785|
| stddev| 8.801267199135454|1.9864582421701988|   null|1502.0359941370607|1501.993958981797| 36.06172819056576|38.583861473580725|
|    min|                 1|                 1|     9E|             10140|            10140|               -63|               -94|
|    max|                31|                 7|     YV|             15376|         

### Sử dụng các method có sẵn để khám phá dữ liệu

Kiểm tra mối quan hệ giữa DepDelay và ArrDelay thông qua hàm `corr`

In [40]:
data.corr("DepDelay", "ArrDelay")

0.9392630367706979

Giá trị tương quan cao, cho thấy 2 biến này cùng tăng hoặc cùng giảm với nhau

### Sử dụng SparkSQL
Sử dụng SparkSQL để hiển thị thời gian delay trung bình theo ngày trong tuần

In [41]:
data.createOrReplaceTempView("flightData")
spark.sql(""" 
SELECT DayOfWeek, CAST(AVG(ArrDelay) as DECIMAL(6,2)) AS `Avg Delay(min)` 
FROM flightData 
GROUP BY DayOfWeek 
ORDER BY DayOfWeek 
""").show()

+---------+--------------+
|DayOfWeek|Avg Delay(min)|
+---------+--------------+
|        1|          7.08|
|        2|          4.39|
|        3|          7.23|
|        4|         10.78|
|        5|          8.71|
|        6|          2.14|
|        7|          5.25|
+---------+--------------+



Ở đây, 2 ngày 4 và 5 cho ra kết quả cao nhất, tương ứng với thứ 5 và thứ 6 trong tuần. Điều này cho thấy vào những ngày cao điểm trong tuần, thời gian trễ chuyến bay cao hơn và tăng

## 3. Chuẩn bị dữ liệu, xây dựng pipeline và huấn luyện mô hình



#### Chuẩn bị dữ liệu 
Để chuẩn dữ liệu, sử dụng dữ liệu đã làm sạch ở phần trước đó.  
Dữ liệu dùng để train sẽ được giữ nguyên 6 cột đầu, cột cuối sẽ ánh xạ thành label, nếu ArrDelay > 15 thì label = 1, ngược lại label = 0

In [42]:
# Import sql functions and ML libraries
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [43]:
data.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [44]:
data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", \
                   ((col("ArrDelay") > 15).cast("Int").alias("label")))

### Chia tập train và tập test
Ở đây chia tập train và test theo tỉ lệ 70:30

In [45]:
splits = data.randomSplit([0.7, 0.3], seed = 42)

train = splits[0]
# rename the target variable in the test set to trueLabel
test = splits[1].withColumnRenamed("label", "trueLabel")

train_rows = train.count()
test_rows = test.count()

print ("Training rows count:", train_rows, " Testing rows count:", test_rows)

Training rows count: 1887560  Testing rows count: 809423


In [46]:
train.show(1, vertical = True)

-RECORD 0----------------
 DayofMonth      | 1     
 DayOfWeek       | 1     
 Carrier         | 9E    
 OriginAirportID | 10423 
 DestAirportID   | 11433 
 DepDelay        | -5    
 label           | 0     
only showing top 1 row



### Chuẩn bị các stage để xử lý dữ liệu

Trong đoạn tiếp theo, SparkML được sử dụng để tạo ra 7 stage. 7 Stages này sẽ được đưa vào 1 pipeline để tạo thành 1 quy trình xử lý
1. **StringIndexer** chuyển các biến định danh thành biến thứ tự
2. **VectorAssembler** gom các biến thứ tự thành 1 vector
3. **VectorIndexer** tạo chỉ mục cho vector chứa biến thứ tự ở stage 2
4. **VectorAssembler** tạo vector chứa giá trị biến liên tục
5. **MinMaxScaler** chuẩn hoá vector ở stage 4
6. **VectorAssembler** tạo vector đặc trưng chứa vector biến thứ tự ở stage 3 và vector chuẩn hoá ở stage 5
7. **LogisticRegression** mô hình phân loại sử dụng logistic regression để phân loại

In [47]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml import Pipeline

#Stage 1. convert string values to indexes for categorical features
strIdx = StringIndexer(inputCol = "Carrier", outputCol = "CarrierIdx")

#Stage 2. combine categorical features into a single vector
catVect = VectorAssembler(inputCols = ["CarrierIdx", "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID"], outputCol="catFeatures")

#Stage 3. create indexes for a vector of categorical features
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")

#Stage 4. create a vector of continuous numeric features
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")

#Stage 5. normalize continuous numeric features
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

#Stage 6. creates a vector of categorical and continuous features
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")

#Stage 7. LogisticRegression classifier that trains a classification model
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)

# Now define the pipeline
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])

### Huấn luyện mô hình
Pipeline được khớp dữ liệu train vào để huấn luyện

In [48]:
import timeit
start_time = timeit.default_timer()

piplineModel = pipeline.fit(train)

elapsed = timeit.default_timer() - start_time

print("Model training complete in:", elapsed, "secs")

Model training complete in: 9.96788129999959 secs


Hiển thị giá trị trung gian giữa các stage trong khi huấn luyện mô hình

In [49]:
piplineModel.transform(train).filter("DayOfWeek == 7").show(2, vertical=True, truncate = 100)

-RECORD 0-------------------------------------------------------------
 DayofMonth      | 1                                                  
 DayOfWeek       | 7                                                  
 Carrier         | 9E                                                 
 OriginAirportID | 10423                                              
 DestAirportID   | 11433                                              
 DepDelay        | -2                                                 
 label           | 0                                                  
 CarrierIdx      | 10.0                                               
 catFeatures     | [10.0,1.0,7.0,10423.0,11433.0]                     
 idxCatFeatures  | [10.0,1.0,6.0,10423.0,11433.0]                     
 numFeatures     | [-2.0]                                             
 normFeatures    | [0.02610966057441253]                              
 features        | [10.0,1.0,6.0,10423.0,11433.0,0.02610966057441253] 
 rawPr

#### Kiểm tra mô hình

Sử dụng hàm transform có sẵn trong mô hình để đưa vào tập test, trả về kết quả các nhãn dự đoán.  
Thực hiện so sánh với nhãn thật (`trueLabel`)

In [50]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(10, truncate=False)

+---------------------------------------------------+----------+---------+
|features                                           |prediction|trueLabel|
+---------------------------------------------------+----------+---------+
|[10.0,1.0,0.0,11057.0,12478.0,0.02558746736292428] |0.0       |0        |
|[10.0,1.0,0.0,11433.0,10423.0,0.02402088772845953] |0.0       |0        |
|[10.0,1.0,0.0,11433.0,14122.0,0.02506527415143603] |0.0       |0        |
|[10.0,1.0,0.0,12339.0,11433.0,0.021409921671018278]|0.0       |0        |
|[10.0,1.0,0.0,12478.0,11278.0,0.02506527415143603] |0.0       |0        |
|[10.0,1.0,0.0,12478.0,12264.0,0.027154046997389034]|0.0       |0        |
|[10.0,1.0,0.0,12478.0,14100.0,0.05430809399477807] |0.0       |1        |
|[10.0,1.0,0.0,13487.0,11193.0,0.02349869451697128] |0.0       |0        |
|[10.0,1.0,0.0,13487.0,14730.0,0.03185378590078329] |0.0       |0        |
|[10.0,1.0,0.0,14122.0,11433.0,0.05378590078328981] |0.0       |1        |
+------------------------

## 4. Đánh giá kết quả và hiệu chỉnh mô hình

#### Đánh giá kết quả: Tính các giá trị trong ma trận lỗi
Để đánh giá kết quả, tính các thang đo sau:  
$TP$: True Positive, khi nhãn dự đoán là 1 và nhãn thật là 1  
$FP$: False Positive, khi nhãn dự đoán là 1 và nhãn thật là 0  
$TN$: True Negative, khi nhãn dự đoán là 0 và nhãn thật là 1  
$FN$: False Negative, khi nhãn dự đoán là 0 và nhãn thật là 0  
Precision = $\frac{TP}{TP+FP}$  
Recall = $\frac{TP}{TP+FN}$


In [51]:
def show_metrics(tp, fp, tn, fn):
    print(f"TP = {tp}")
    print(f"FP = {fp}")
    print(f"TN = {tn}")
    print(f"FN = {fn}")
    print(f"Precision = {tp / (tp + fp)}")
    print(f"Recall = {tp / (tp + fn)}")

In [52]:
tp = float(predicted.filter("prediction == 1.0 AND trueLabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND trueLabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND trueLabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND trueLabel == 1").count())
show_metrics(tp, fp, tn, fn)

TP = 19277.0
FP = 81.0
TN = 647526.0
FN = 142539.0
Precision = 0.9958156834383717
Recall = 0.11912913432540663


#### Tính giá trị AUC
Sử dụng BinaryClassificationEvaluator để tính AUC. Giá trị AUC càng gần 1 thì mô hình càng dự đoán không ngẫu nhiên, có cơ sở

In [53]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("Area under the ROC curve = ", aur)

Area under the ROC curve =  0.9230114851471186


#### Hiệu chỉnh mô hình

Giá trị ngưỡng mặc định của LogisticRegression là 0.5, tức là khi tính toán dựa vào các trọng số, nếu ra kết quả > 0.5 thì nhãn dự đoán là 1, ngược lại là 0. Ta thấy ở trên Recall khá thấp, vậy nên để tăng Recall thì phải giảm FN xuống bằng cách hạ threshold xuống 0.35

In [54]:
#Change the threshold to 0.3 and create a new LogisticRegression model
lr2 = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3, threshold=0.35)

#Set up new pipeline
pipeline2 = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr2])
model2 = pipeline2.fit(train)

#Make new predictions
newPrediction = model2.transform(test)
newPrediction.select("rawPrediction", "probability", "prediction", "trueLabel")\
.show(10, truncate=False)

+----------------------------------------+----------------------------------------+----------+---------+
|rawPrediction                           |probability                             |prediction|trueLabel|
+----------------------------------------+----------------------------------------+----------+---------+
|[1.6028410930250616,-1.6028410930250616]|[0.83241509258929,0.16758490741070997]  |0.0       |0        |
|[1.6472109887757473,-1.6472109887757473]|[0.8385137512235306,0.16148624877646944]|0.0       |0        |
|[1.620881286552173,-1.620881286552173]  |[0.8349166341374121,0.16508336586258787]|0.0       |0        |
|[1.7256164253104487,-1.7256164253104487]|[0.8488508537665199,0.15114914623348008]|0.0       |0        |
|[1.6287422788121078,-1.6287422788121078]|[0.8359972707280444,0.16400272927195558]|0.0       |0        |
|[1.5731771293119245,-1.5731771293119245]|[0.8282360615979741,0.17176393840202586]|0.0       |0        |
|[0.845853440305421,-0.845853440305421]  |[0.6996965841

In [55]:
# Recalculate confusion matrix, using the new predictions
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())

show_metrics(tp2, fp2, tn2, fn2)

TP = 42021.0
FP = 134.0
TN = 647473.0
FN = 119795.0
Precision = 0.996821254892658
Recall = 0.2596838384337766


Chú ý rằng FN đã giảm và FP đã tăng

#### Hiệu chỉnh mô hình với CrossValidator

Sử dụng kỹ thuật CrossValidation và ParamGridBuilder để tạo ra các khoảng của các tham số của bộ phân loại LR. Khi đó, Cross Validator sẽ tạo ra các pipeline và thay đổi từng tham số trong khoảng được định nghĩa trong ParamGridBuilder, nhằm tìm ra tham số tốt nhất.

In [56]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

paramGrid = ParamGridBuilder()\
.addGrid(lr.regParam, [0.3])\
.addGrid(lr.maxIter, [10])\
.addGrid(lr.threshold, [0.25, 0.3, 0.35])\
.build()

cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(),\
                    estimatorParamMaps=paramGrid, numFolds=5)

modelCV = cv.fit(train)

#### Kiểm tra mô hình trên tập test


In [57]:
predictionCV = modelCV.transform(test)
predictedCV = predictionCV.select("features", "prediction", "trueLabel")
predictedCV.show(10, truncate=False)

+---------------------------------------------------+----------+---------+
|features                                           |prediction|trueLabel|
+---------------------------------------------------+----------+---------+
|[10.0,1.0,0.0,11057.0,12478.0,0.02558746736292428] |0.0       |0        |
|[10.0,1.0,0.0,11433.0,10423.0,0.02402088772845953] |0.0       |0        |
|[10.0,1.0,0.0,11433.0,14122.0,0.02506527415143603] |0.0       |0        |
|[10.0,1.0,0.0,12339.0,11433.0,0.021409921671018278]|0.0       |0        |
|[10.0,1.0,0.0,12478.0,11278.0,0.02506527415143603] |0.0       |0        |
|[10.0,1.0,0.0,12478.0,12264.0,0.027154046997389034]|0.0       |0        |
|[10.0,1.0,0.0,12478.0,14100.0,0.05430809399477807] |1.0       |1        |
|[10.0,1.0,0.0,13487.0,11193.0,0.02349869451697128] |0.0       |0        |
|[10.0,1.0,0.0,13487.0,14730.0,0.03185378590078329] |0.0       |0        |
|[10.0,1.0,0.0,14122.0,11433.0,0.05378590078328981] |1.0       |1        |
+------------------------

In [58]:
# Recalculate confusion matrix, using the new predictions
tp3 = float(predictionCV.filter("prediction == 1.0 AND truelabel == 1").count())
fp3 = float(predictionCV.filter("prediction == 1.0 AND truelabel == 0").count())
tn3 = float(predictionCV.filter("prediction == 0.0 AND truelabel == 0").count())
fn3 = float(predictionCV.filter("prediction == 0.0 AND truelabel == 1").count())

show_metrics(tp3, fp3, tn3, fn3)

TP = 86496.0
FP = 1574.0
TN = 646033.0
FN = 75320.0
Precision = 0.9821278528443284
Recall = 0.5345330498838187


Recall đã tăng lên 50%

In [59]:
bestPipeline = modelCV.bestModel
bestLRModel = bestPipeline.stages[6]
bestParams = bestLRModel.extractParamMap()

In [60]:
#type(bestParams)
for k,v in bestParams.items():
    print("Key: ", k, " ---> Value = ", v)

Key:  LogisticRegression_cd1c8d81c1ef__aggregationDepth  ---> Value =  2
Key:  LogisticRegression_cd1c8d81c1ef__elasticNetParam  ---> Value =  0.0
Key:  LogisticRegression_cd1c8d81c1ef__family  ---> Value =  auto
Key:  LogisticRegression_cd1c8d81c1ef__featuresCol  ---> Value =  features
Key:  LogisticRegression_cd1c8d81c1ef__fitIntercept  ---> Value =  True
Key:  LogisticRegression_cd1c8d81c1ef__labelCol  ---> Value =  label
Key:  LogisticRegression_cd1c8d81c1ef__maxBlockSizeInMB  ---> Value =  0.0
Key:  LogisticRegression_cd1c8d81c1ef__maxIter  ---> Value =  10
Key:  LogisticRegression_cd1c8d81c1ef__predictionCol  ---> Value =  prediction
Key:  LogisticRegression_cd1c8d81c1ef__probabilityCol  ---> Value =  probability
Key:  LogisticRegression_cd1c8d81c1ef__rawPredictionCol  ---> Value =  rawPrediction
Key:  LogisticRegression_cd1c8d81c1ef__regParam  ---> Value =  0.3
Key:  LogisticRegression_cd1c8d81c1ef__standardization  ---> Value =  True
Key:  LogisticRegression_cd1c8d81c1ef__thres

Các tham số tốt nhất cho mô hình LogisticRegression sau khi áp dụng CrossValidator để tìm tham số  
Threshold tốt nhất là 0.30

In [61]:
eval2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur2 = eval2.evaluate(predictionCV)
print ("Area under the ROC curve = ", aur2)

Area under the ROC curve =  0.9230119021256126


Không có sự thay đổi lớn của AUC